# 02-SQL-Consultas-Intermedias

In [4]:
# Librerias
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# Conerctar a mi SQLserver
driver = "ODBC Driver 17 for SQL Server"
server = r"DESKTOP-GB9FFBV"
database = "BD_SQL_"

In [6]:
# Crear motor de conexión
connection_string = f"mssql+pyodbc://@{server}/{database}?driver={driver}"
engine = create_engine(connection_string)

In [7]:
# -------------------------------
# 1. Obtener todos los empleados cuyo salario sea mayor al promedio general.
# -------------------------------
df1 = pd.read_sql("""
SELECT id_vendedor, nombre, salario
FROM empleados
WHERE salario > (SELECT AVG(salario) AS avg_general FROM empleados)
ORDER BY salario;
""", engine)
print(df1)

    id_vendedor       nombre      salario
0            49  Empleado 49  5142.859863
1            17  Empleado 17  5536.040039
2            22  Empleado 22  5621.919922
3            33  Empleado 33  5678.729980
4            16  Empleado 16  5732.459961
5            40  Empleado 40  5745.560059
6            45  Empleado 45  5822.399902
7            48  Empleado 48  5859.759766
8            24  Empleado 24  5895.149902
9            26  Empleado 26  5961.129883
10           34  Empleado 34  6011.979980
11            8   Empleado 8  6128.089844
12           15  Empleado 15  6367.370117
13           43  Empleado 43  6479.419922
14           44  Empleado 44  6624.910156
15            1   Empleado 1  6840.640137
16            9   Empleado 9  6868.859863
17            5   Empleado 5  6883.620117
18           31  Empleado 31  6940.770020
19           19  Empleado 19  6969.410156
20           29  Empleado 29  7062.290039
21           42  Empleado 42  7162.830078
22           47  Empleado 47  7396

In [8]:
# -------------------------------
# 2. Listar los clientes que realizaron más de 5 compras en el último mes.
# -------------------------------
df2 = pd.read_sql("""
WITH t1 AS (
    SELECT v.id_cliente, c.nombre, COUNT(v.id_venta) AS num_compras
    FROM clientes c INNER JOIN ventas v ON c.id_cliente = v.id_cliente
    WHERE v.fecha_venta > DATEADD(month,-1,DATETRUNC(month, SYSDATETIME()))
      AND v.fecha_venta < DATETRUNC(month, SYSDATETIME())
    GROUP BY v.id_cliente, c.nombre
)
SELECT id_cliente, nombre, num_compras
FROM t1
WHERE num_compras > 5;
""", engine)
print(df2)

Empty DataFrame
Columns: [id_cliente, nombre, num_compras]
Index: []


In [9]:
# -------------------------------
# 3. Mostrar los productos que nunca fueron vendidos.
# -------------------------------
df3 = pd.read_sql("""
SELECT p.id_producto, p.nombre_producto
FROM productos p LEFT JOIN ventas v ON p.id_producto = v.id_producto
WHERE v.id_producto IS NULL;
""", engine)
print(df3)

Empty DataFrame
Columns: [id_producto, nombre_producto]
Index: []


In [10]:
# -------------------------------
# 4. Calcular el salario promedio por departamento.
# -------------------------------
df4 = pd.read_sql("""
SELECT d.id_departamento, d.nombre_departamento, AVG(e.salario) AS avg_salario
FROM departamentos d INNER JOIN empleados e ON d.id_departamento = e.id_departamento
GROUP BY d.id_departamento, d.nombre_departamento;
""", engine)
print(df4)

    id_departamento nombre_departamento  avg_salario
0                 1             Depto 1  5875.004883
1                 2             Depto 2  2221.429932
2                 3             Depto 3  5779.280111
3                 4             Depto 4  3066.270060
4                 5             Depto 5  5457.135010
5                 6             Depto 6  6882.070068
6                 8             Depto 8  6868.859863
7                 9             Depto 9  4806.419922
8                11            Depto 11  4853.114990
9                12            Depto 12  6969.410156
10               13            Depto 13  3613.179932
11               16            Depto 16  7864.180176
12               19            Depto 19  5796.109863
13               20            Depto 20  1715.729980
14               21            Depto 21  2320.719971
15               25            Depto 25  4280.459961
16               26            Depto 26  5745.560059
17               27            Depto 27  5383.

In [11]:
# -------------------------------
# 5. Listar los empleados contratados en el mismo año que su jefe.
# -------------------------------
df5 = pd.read_sql("""
SELECT e.id_vendedor AS id_empleado, e.nombre AS nombre_empleado, e.fecha_contratacion AS fecha_empleado,
       j.id_vendedor AS id_jefe, j.nombre AS nombre_jefe, j.fecha_contratacion AS fecha_jefe
FROM empleados e INNER JOIN empleados j ON e.id_jefe = j.id_vendedor
WHERE DATEPART(year, e.fecha_contratacion) = DATEPART(year, j.fecha_contratacion);
""", engine)
print(df5)

Empty DataFrame
Columns: [id_empleado, nombre_empleado, fecha_empleado, id_jefe, nombre_jefe, fecha_jefe]
Index: []


In [12]:
# -------------------------------
# 6. Encontrar las ventas cuyo monto sea mayor que el promedio del mismo mes.
# -------------------------------
df6 = pd.read_sql("""
WITH t1 AS (
    SELECT DATEPART(year, fecha_venta) AS anio, DATEPART(month, fecha_venta) AS mes,
           AVG(precio_unitario * cantidad) AS avg_mes
    FROM ventas
    GROUP BY DATEPART(year, fecha_venta), DATEPART(month, fecha_venta)
)
SELECT t1.anio, t1.mes, t1.avg_mes, v.id_venta, v.precio_unitario * cantidad AS monto
FROM ventas v INNER JOIN t1
  ON DATEPART(year, fecha_venta) = anio AND DATEPART(month, fecha_venta) = mes
WHERE precio_unitario * cantidad > avg_mes;
""", engine)
print(df6)

       anio  mes     avg_mes  id_venta        monto
0      2023    4  798.296046         2  1694.450073
1      2023    9  782.426063         6   897.239960
2      2023    8  779.208368         7  1225.890015
3      2023    5  803.690606         8   971.149979
4      2025    1  772.034230         9  1780.899963
...     ...  ...         ...       ...          ...
13622  2023    2  763.034201     32755   842.200012
13623  2023    1  755.260636     32762  2178.699951
13624  2025    3  757.197646     32763   804.280029
13625  2023   12  797.412348     32764  2253.000031
13626  2025    4  736.748221     32766   751.200012

[13627 rows x 5 columns]


In [13]:
# -------------------------------
# 7. Mostrar los clientes que no tienen pedidos registrados.
# -------------------------------
df7 = pd.read_sql("""
SELECT c.id_cliente, c.nombre
FROM clientes c LEFT JOIN ventas v ON c.id_cliente = v.id_cliente
WHERE v.id_cliente IS NULL;
""", engine)
print(df7)

# -------------------------

Empty DataFrame
Columns: [id_cliente, nombre]
Index: []


In [15]:
# -------------------------------
# 8. Listar los productos cuyo precio lista está por encima del promedio de la categoría.
# -------------------------------
df8 = pd.read_sql("""
WITH t1 AS (
    SELECT categoria, AVG(precio_lista) AS avg_pl_catg
    FROM productos
    GROUP BY categoria
)
SELECT p.categoria, t1.avg_pl_catg, p.id_producto, p.nombre_producto, p.precio_lista
FROM productos p INNER JOIN t1 ON p.categoria = t1.categoria
WHERE p.precio_lista > t1.avg_pl_catg
ORDER BY p.id_producto DESC;
""", engine)
print(df8)

      categoria  avg_pl_catg  id_producto nombre_producto  precio_lista
0         Hogar   278.107277           39     Producto 39    424.130005
1          Ropa   241.755554           38     Producto 38    438.910004
2   Electronica   258.197139           35     Producto 35    471.529999
3      Juguetes   252.294614           33     Producto 33    268.679993
4          Ropa   241.755554           31     Producto 31    419.209991
5         Hogar   278.107277           25     Producto 25    302.130005
6      Juguetes   252.294614           24     Producto 24    294.500000
7      Juguetes   252.294614           22     Producto 22    423.109985
8      Juguetes   252.294614           19     Producto 19    397.109985
9   Electronica   258.197139           18     Producto 18    325.369995
10         Ropa   241.755554           16     Producto 16    328.429993
11        Hogar   278.107277           15     Producto 15    467.890015
12  Electronica   258.197139           14     Producto 14    499

In [16]:
# -------------------------------
# 9. Mostrar los empleados con el segundo salario más alto por departamento.
# -------------------------------
df9 = pd.read_sql("""
WITH t1 AS (
    SELECT id_departamento, id_vendedor, nombre, salario,
           DENSE_RANK() OVER(PARTITION BY id_departamento ORDER BY salario DESC) AS ranking_salarios
    FROM empleados
)
SELECT id_departamento, id_vendedor, nombre, salario
FROM t1
WHERE ranking_salarios = 2;
""", engine)
print(df9)

    id_departamento  id_vendedor       nombre      salario
0                 1           22  Empleado 22  5621.919922
1                 3           34  Empleado 34  6011.979980
2                 4           36  Empleado 36  3484.520020
3                 5           32  Empleado 32  3315.290039
4                 6           15  Empleado 15  6367.370117
5                 9           18  Empleado 18  3790.439941
6                11           20  Empleado 20  4563.370117
7                19           16  Empleado 16  5732.459961
8                25           37  Empleado 37  2882.189941
9                27            3   Empleado 3  3825.689941
10               28           28  Empleado 28  2636.729980
11               30           14  Empleado 14  4427.279785
12               32           11  Empleado 11  3059.179932
13               33            6   Empleado 6  3678.850098
14               34           39  Empleado 39  3475.649902
15               37           10  Empleado 10  2154.1398

In [17]:
# -------------------------------
# 10. Calcular el total de ventas por año y mes.
# -------------------------------
df10 = pd.read_sql("""
SELECT DATEPART(year, fecha_venta) AS anio, DATEPART(month, fecha_venta) AS mes,
       SUM(precio_unitario * cantidad) AS total_periodo_mes
FROM ventas
GROUP BY DATEPART(year, fecha_venta), DATEPART(month, fecha_venta)
ORDER BY anio, mes;
""", engine)
print(df10)

    anio  mes  total_periodo_mes
0   2023    1      783960.540022
1   2023    2      731749.799223
2   2023    3      862632.819351
3   2023    4      874134.170298
4   2023    5      915403.600279
5   2023    6      837752.710169
6   2023    7      844590.089632
7   2023    8      801805.410563
8   2023    9      776949.080631
9   2023   10      842199.071150
10  2023   11      799602.931625
11  2023   12      811765.770716
12  2024    1      846446.709148
13  2024    2      778114.340755
14  2024    3      862058.439325
15  2024    4      829257.689625
16  2024    5      834017.860613
17  2024    6      755149.479601
18  2024    7      861960.560270
19  2024    8      858511.480249
20  2024    9      800068.659338
21  2024   10      828476.979780
22  2024   11      815658.539993
23  2024   12      902896.990057
24  2025    1      843061.379539
25  2025    2      824561.499800
26  2025    3      817773.457611
27  2025    4      766218.149401
28  2025    5      809648.369841
29  2025  

In [18]:
# -------------------------------
# 11. Encontrar los empleados que ganan igual que otro empleado (salarios duplicados).
# -------------------------------
df11 = pd.read_sql("""
SELECT e1.id_vendedor, e1.nombre
FROM empleados e1 INNER JOIN empleados e2 ON e1.id_vendedor <> e2.id_vendedor
WHERE e1.salario = e2.salario;
""", engine)
print(df11)

Empty DataFrame
Columns: [id_vendedor, nombre]
Index: []


In [19]:
# -------------------------------
# 12. Mostrar los productos vendidos al menos en 3 países distintos.
# -------------------------------
df12 = pd.read_sql("""
WITH t1 AS (
    SELECT p.id_producto, p.nombre_producto, COUNT(DISTINCT c.pais) AS num_paises
    FROM ventas v INNER JOIN clientes c ON v.id_cliente = c.id_cliente
                  INNER JOIN productos p ON v.id_producto = p.id_producto
    GROUP BY p.id_producto, p.nombre_producto
)
SELECT id_producto, nombre_producto, num_paises
FROM t1
WHERE num_paises >= 3;
""", engine)
print(df12)

    id_producto nombre_producto  num_paises
0             1      Producto 1           4
1             2      Producto 2           4
2             3      Producto 3           4
3             4      Producto 4           4
4             5      Producto 5           4
5             6      Producto 6           4
6             7      Producto 7           4
7             8      Producto 8           4
8             9      Producto 9           4
9            10     Producto 10           4
10           11     Producto 11           4
11           12     Producto 12           4
12           13     Producto 13           4
13           14     Producto 14           4
14           15     Producto 15           4
15           16     Producto 16           4
16           17     Producto 17           4
17           18     Producto 18           4
18           19     Producto 19           4
19           20     Producto 20           4
20           21     Producto 21           4
21           22     Producto 22 

In [20]:
# -------------------------------
# 13. Listar los pedidos que incluyen más de 5 productos diferentes.
# -------------------------------
df13 = pd.read_sql("""
WITH t1 AS (
    SELECT id_venta, COUNT(DISTINCT id_producto) AS num_productos
    FROM ventas
    GROUP BY id_venta
)
SELECT id_venta, num_productos
FROM t1
WHERE num_productos > 5;
""", engine)
print(df13)

Empty DataFrame
Columns: [id_venta, num_productos]
Index: []


In [21]:
# -------------------------------
# 14. Mostrar los departamentos que no tienen empleados.
# -------------------------------
df14 = pd.read_sql("""
SELECT d.id_departamento, d.nombre_departamento
FROM departamentos d LEFT JOIN empleados e ON d.id_departamento = e.id_departamento
WHERE e.id_departamento IS NULL;
""", engine)
print(df14)

    id_departamento nombre_departamento
0                 7             Depto 7
1                10            Depto 10
2                14            Depto 14
3                15            Depto 15
4                17            Depto 17
5                18            Depto 18
6                22            Depto 22
7                23            Depto 23
8                24            Depto 24
9                36            Depto 36
10               39            Depto 39


In [22]:
# -------------------------------
# 15. Obtener el cliente con la compra más alta del año.
# -------------------------------
df15 = pd.read_sql("""
WITH t1 AS (
    SELECT DATEPART(year, fecha_venta) AS anio, id_cliente,
           SUM(precio_unitario * cantidad) AS total_venta,
           DENSE_RANK() OVER (PARTITION BY DATEPART(year, fecha_venta)
                              ORDER BY SUM(precio_unitario * cantidad) DESC) AS ranking
    FROM ventas
    GROUP BY DATEPART(year, fecha_venta), id_cliente
)
SELECT *
FROM t1
WHERE ranking = 1;
""", engine)
print(df15)

   anio  id_cliente   total_venta  ranking
0  2023         179  27163.660080        1
1  2024         685  24545.099792        1
2  2025         397  15943.690159        1


In [23]:
# -------------------------------
# 16. Calcular la diferencia entre el salario más alto y el más bajo de cada departamento.
# -------------------------------
df16 = pd.read_sql("""
SELECT d.id_departamento, d.nombre_departamento,
       MAX(salario) - MIN(salario) AS dif_salarios
FROM departamentos d LEFT JOIN empleados e ON d.id_departamento = e.id_departamento
GROUP BY d.id_departamento, d.nombre_departamento;
""", engine)
print(df16)

    id_departamento nombre_departamento  dif_salarios
0                 1             Depto 1    506.169922
1                 2             Depto 2      0.000000
2                 3             Depto 3   2355.419922
3                 4             Depto 4   1808.950073
4                 5             Depto 5   4283.689941
5                 6             Depto 6   1029.399902
6                 7             Depto 7           NaN
7                 8             Depto 8      0.000000
8                 9             Depto 9   2031.959961
9                10            Depto 10           NaN
10               11            Depto 11    579.489746
11               12            Depto 12      0.000000
12               13            Depto 13      0.000000
13               14            Depto 14           NaN
14               15            Depto 15           NaN
15               16            Depto 16      0.000000
16               17            Depto 17           NaN
17               18         

In [24]:
# -------------------------------
# 17. Mostrar los empleados cuyo nombre comienza con 'A' y terminan con 'Z'.
# -------------------------------
df17 = pd.read_sql("SELECT * FROM empleados WHERE nombre LIKE 'A%Z';", engine)
print(df17)

Empty DataFrame
Columns: [id_vendedor, nombre, cargo, id_jefe, id_departamento, fecha_contratacion, salario]
Index: []


In [25]:
# -------------------------------
# 18. Listar los productos que no se vendieron en los últimos 6 meses.
# -------------------------------
df18 = pd.read_sql("""
WITH t1 AS (
    SELECT DISTINCT id_producto
    FROM ventas
    WHERE fecha_venta >= DATEADD(month, -6, DATETRUNC(month, SYSDATETIME()))
      AND fecha_venta < DATETRUNC(month, SYSDATETIME())
)
SELECT p.id_producto, p.nombre_producto
FROM productos p LEFT JOIN t1 ON p.id_producto = t1.id_producto
WHERE t1.id_producto IS NULL;
""", engine)
print(df18)

Empty DataFrame
Columns: [id_producto, nombre_producto]
Index: []


In [26]:
# -------------------------------
# 19. Obtener los tres clientes con más pedidos totales.
# -------------------------------
df19 = pd.read_sql("""
WITH t1 AS (
    SELECT c.id_cliente, c.nombre,
           SUM(v.precio_unitario * v.cantidad) AS total,
           DENSE_RANK() OVER (ORDER BY SUM(v.precio_unitario * v.cantidad) DESC) AS ranking
    FROM clientes c INNER JOIN ventas v ON c.id_cliente = v.id_cliente
    GROUP BY c.id_cliente, c.nombre
)
SELECT * FROM t1 WHERE ranking <= 3;
""", engine)
print(df19)

   id_cliente       nombre         total  ranking
0         685  Cliente 685  51230.529732        1
1         343  Cliente 343  49909.009983        2
2          16   Cliente 16  49086.589737        3


In [27]:
# -------------------------------
# 20. Mostrar el promedio de ventas por producto.
# -------------------------------
df20 = pd.read_sql("""
SELECT p.id_producto, p.nombre_producto,
       AVG(v.precio_unitario * v.cantidad) AS avg_compras
FROM productos p INNER JOIN ventas v ON p.id_producto = v.id_producto
GROUP BY p.id_producto, p.nombre_producto;
""", engine)
print(df20)

    id_producto nombre_producto  avg_compras
0            23     Producto 23   792.004756
1            29     Producto 29   792.693154
2            15     Producto 15   773.631189
3             9      Producto 9   756.672545
4             3      Producto 3   812.625318
5            32     Producto 32   781.620711
6            26     Producto 26   805.391167
7            12     Producto 12   762.415848
8            35     Producto 35   764.646717
9             6      Producto 6   778.972546
10           21     Producto 21   762.620150
11           27     Producto 27   775.273924
12           38     Producto 38   828.149847
13            7      Producto 7   752.991349
14            1      Producto 1   799.680535
15           24     Producto 24   754.477519
16           18     Producto 18   782.763903
17           30     Producto 30   781.456897
18           10     Producto 10   755.387422
19            4      Producto 4   762.706860
20           19     Producto 19   787.137076
21        

In [28]:
# -------------------------------
# 21. Calcular el porcentaje que representa cada categoría sobre las ventas totales.
# -------------------------------
df21 = pd.read_sql("""
SELECT p.categoria,
       SUM(v.precio_unitario * v.cantidad) / SUM(SUM(v.precio_unitario * v.cantidad)) OVER () * 100.0 AS Parti
FROM productos p INNER JOIN ventas v ON p.id_producto = v.id_producto
GROUP BY p.categoria
ORDER BY Parti DESC;
""", engine)
print(df21)

     categoria      Parti
0     Juguetes  32.484312
1        Hogar  27.499429
2         Ropa  22.902743
3  Electronica  17.113516


In [29]:
# -------------------------------
# 22. Mostrar los empleados que ganan más que su jefe.
# -------------------------------
df22 = pd.read_sql("""
SELECT *
FROM empleados j INNER JOIN empleados e ON j.id_jefe = e.id_vendedor
WHERE j.salario > e.salario;
""", engine)
print(df22)

Empty DataFrame
Columns: [id_vendedor, nombre, cargo, id_jefe, id_departamento, fecha_contratacion, salario, id_vendedor, nombre, cargo, id_jefe, id_departamento, fecha_contratacion, salario]
Index: []


In [30]:
# -------------------------------
# 23. Listar los pedidos entregados fuera de plazo.
# -------------------------------
df23 = pd.read_sql("SELECT * FROM ventas WHERE estado_entrega = 'FUERA_DE_PLAZO';", engine)
print(df23)

Empty DataFrame
Columns: [id_venta, id_cliente, id_producto, id_vendedor, fecha_venta, cantidad, precio_unitario, canal, region, estado_entrega]
Index: []


In [31]:
# -------------------------------
# 24. Obtener la cantidad de clientes por país.
# -------------------------------
df24 = pd.read_sql("SELECT pais, COUNT(DISTINCT id_cliente) AS num_clientes FROM clientes GROUP BY pais;", engine)
print(df24)

       pais  num_clientes
0    Mexico           227
1      Peru           199
2  Colombia           187
3     Chile           187


In [32]:
# -------------------------------
# 25. Calcular el total de ingresos por canal de venta.
# -------------------------------
df25 = pd.read_sql("""
SELECT canal, SUM(precio_unitario * cantidad) AS ingresos_por_canal
FROM ventas
GROUP BY canal;
""", engine)
print(df25)

    canal  ingresos_por_canal
0  TIENDA        8.519360e+06
1     WEB        8.533593e+06
2     APP        8.466682e+06


In [33]:
# -------------------------------
# 26. Mostrar los 2 productos más vendidos por región.
# -------------------------------
df26 = pd.read_sql("""
WITH t1 AS (
    SELECT region, id_producto, SUM(cantidad) AS cantidad_vendida,
           DENSE_RANK() OVER (PARTITION BY region ORDER BY SUM(cantidad) DESC) AS ranking
    FROM ventas
    GROUP BY region, id_producto
)
SELECT * FROM t1 WHERE ranking <= 2;
""", engine)
print(df26)

   region  id_producto  cantidad_vendida  ranking
0  Centro           36               997        1
1  Centro           22               915        2
2   Norte            2               910        1
3   Norte           21               905        2
4     Sur           30               941        1
5     Sur           34               916        2


In [34]:
# -------------------------------
# 27. Listar los empleados que llevan más de 10 años en la empresa.
# -------------------------------
df27 = pd.read_sql("""
SELECT * FROM empleados
WHERE fecha_contratacion < DATEADD(year, -10, SYSDATETIME());
""", engine)
print(df27)

Empty DataFrame
Columns: [id_vendedor, nombre, cargo, id_jefe, id_departamento, fecha_contratacion, salario]
Index: []


In [35]:
# -------------------------------
# 28. Mostrar los clientes que compraron productos de todas las categorías.
# -------------------------------
df28 = pd.read_sql("""
WITH t1 AS (
    SELECT v.id_cliente, COUNT(DISTINCT p.categoria) AS cant_categorias
    FROM productos p INNER JOIN ventas v ON p.id_producto = v.id_producto
    GROUP BY v.id_cliente
)
SELECT * FROM t1
WHERE cant_categorias = (SELECT COUNT(DISTINCT categoria) AS cant_total_catego FROM productos);
""", engine)
print(df28)

     id_cliente  cant_categorias
0           261                4
1           593                4
2           687                4
3           355                4
4            23                4
..          ...              ...
794         592                4
795         100                4
796         443                4
797         541                4
798         492                4

[799 rows x 2 columns]


In [36]:
# -------------------------------
# 29. Calcular el promedio móvil de ventas de los últimos 3 meses.
# -------------------------------
df29 = pd.read_sql("""
WITH t1 AS (
    SELECT DATEPART(year, fecha_venta) AS anio, DATEPART(month, fecha_venta) AS mes,
           SUM(precio_unitario * cantidad) AS total
    FROM ventas
    GROUP BY DATEPART(year, fecha_venta), DATEPART(month, fecha_venta)
)
SELECT anio, mes, total,
       AVG(total) OVER (ORDER BY anio, mes ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS prom
FROM t1;
""", engine)
print(df29)

    anio  mes          total           prom
0   2023    1  783960.540022  783960.540022
1   2023    2  731749.799223  757855.169622
2   2023    3  862632.819351  792781.052865
3   2023    4  874134.170298  822838.929624
4   2023    5  915403.600279  884056.863309
5   2023    6  837752.710169  875763.493582
6   2023    7  844590.089632  865915.466693
7   2023    8  801805.410563  828049.403454
8   2023    9  776949.080631  807781.526942
9   2023   10  842199.071150  806984.520781
10  2023   11  799602.931625  806250.361135
11  2023   12  811765.770716  817855.924497
12  2024    1  846446.709148  819271.803830
13  2024    2  778114.340755  812108.940207
14  2024    3  862058.439325  828873.163076
15  2024    4  829257.689625  823143.489902
16  2024    5  834017.860613  841777.996521
17  2024    6  755149.479601  806141.676613
18  2024    7  861960.560270  817042.633495
19  2024    8  858511.480249  825207.173374
20  2024    9  800068.659338  840180.233286
21  2024   10  828476.979780  82

In [37]:
# -------------------------------
# 30. Mostrar la diferencia en ventas entre el mes actual y el mes anterior.
# -------------------------------
df30 = pd.read_sql("""
WITH t1 AS (
    SELECT DATEPART(year, fecha_venta) AS anio, DATEPART(month, fecha_venta) AS mes,
           SUM(precio_unitario * cantidad) AS total
    FROM ventas
    GROUP BY DATEPART(year, fecha_venta), DATEPART(month, fecha_venta)
)
SELECT anio, mes, total - LAG(total) OVER (ORDER BY anio, mes) AS diferencia
FROM t1;
""", engine)
print(df30)

    anio  mes     diferencia
0   2023    1            NaN
1   2023    2  -52210.740799
2   2023    3  130883.020128
3   2023    4   11501.350946
4   2023    5   41269.429981
5   2023    6  -77650.890110
6   2023    7    6837.379463
7   2023    8  -42784.679070
8   2023    9  -24856.329931
9   2023   10   65249.990519
10  2023   11  -42596.139524
11  2023   12   12162.839090
12  2024    1   34680.938433
13  2024    2  -68332.368393
14  2024    3   83944.098570
15  2024    4  -32800.749701
16  2024    5    4760.170988
17  2024    6  -78868.381012
18  2024    7  106811.080669
19  2024    8   -3449.080021
20  2024    9  -58442.820911
21  2024   10   28408.320442
22  2024   11  -12818.439787
23  2024   12   87238.450064
24  2025    1  -59835.610518
25  2025    2  -18499.879740
26  2025    3   -6788.042189
27  2025    4  -51555.308210
28  2025    5   43430.220440
29  2025    6  -29473.270287
30  2025    7   42857.750408
